In [2]:
import pandas as pd
import numpy as np

----
IQQM CSV  output

In [3]:
input=pd.read_csv(filepath_or_buffer="./tests/35013f15b(GIS).csv",quotechar='\'')
temp=input
temp

,Reach,Branch,Link,Node number,Downstream node,Node type,Longitude,Latitude,Label,Associated node
0,Reach001,1,1,129,136,1.0,150.9459,-31.6886,Barnard River,0
1,Reach001,1,2,136,132,3.1,150.9402,-31.7452,Macquarie Gen transfer to Hunter,143
2,Reach001,2,1,131,132,1.0,151.0374,-31.7469,Orham Creek,0
3,Reach001,3,1,132,133,11.0,150.9761,-31.8026,Barnard R / Orham Ck,0
4,Reach001,3,2,133,134,0.0,150.9762,-31.8331,Barnard R GS for 80% flow,0
...,...,...,...,...,...,...,...,...,...,...
386,Reach096,64,1,678,679,1.2,152.0447,-33.1384,Receive transfers from Lostock,0
387,Reach096,64,2,679,684,2.1,152.1403,-33.1564,HWC Account in Glennies,0
388,Reach096,64,3,684,681,3.0,152.2281,-33.1456,Diversion from HWC Reserve,0
389,Reach096,65,1,681,610,11.0,152.3344,-33.1150,day count confluence,0


In [4]:
nodes=[{'data': {'id': str(temp["Node number"][x]), 'label': temp["Label"][x]}, 'position': {'x' : temp["Longitude"][x]*4000, 'y': temp["Latitude"][x]*-4000}} for x in range(len(temp))]

In [5]:
elements=[{'data': {'source': str(temp["Node number"][x]), 'target': str(temp["Downstream node"][x])}} for x in range(len(temp)-1) if str(temp["Downstream node"][x]) not in ['1008','1010','1014']]

In [6]:
[x["data"] for x in elements]

[{'source': '129', 'target': '136'},
 {'source': '136', 'target': '132'},
 {'source': '131', 'target': '132'},
 {'source': '132', 'target': '133'},
 {'source': '133', 'target': '134'},
 {'source': '134', 'target': '135'},
 {'source': '1', 'target': '135'},
 {'source': '135', 'target': '219'},
 {'source': '219', 'target': '2'},
 {'source': '2', 'target': '140'},
 {'source': '143', 'target': '530'},
 {'source': '530', 'target': '141'},
 {'source': '141', 'target': '502'},
 {'source': '502', 'target': '510'},
 {'source': '510', 'target': '130'},
 {'source': '130', 'target': '138'},
 {'source': '138', 'target': '140'},
 {'source': '140', 'target': '152'},
 {'source': '152', 'target': '3'},
 {'source': '3', 'target': '215'},
 {'source': '215', 'target': '4'},
 {'source': '4', 'target': '80'},
 {'source': '5', 'target': '220'},
 {'source': '220', 'target': '80'},
 {'source': '80', 'target': '81'},
 {'source': '6', 'target': '221'},
 {'source': '221', 'target': '81'},
 {'source': '81', 'targe

In [7]:
[x["data"] for x in nodes]

[{'id': '129', 'label': 'Barnard River'},
 {'id': '136', 'label': 'Macquarie Gen transfer to Hunter'},
 {'id': '131', 'label': 'Orham Creek'},
 {'id': '132', 'label': 'Barnard R / Orham Ck'},
 {'id': '133', 'label': 'Barnard R GS for 80% flow'},
 {'id': '134', 'label': 'remove d/s flows'},
 {'id': '1', 'label': 'Glenbawn inflow'},
 {'id': '135', 'label': 'Barnard / Hunter'},
 {'id': '219', 'label': 'Glenbawn Irr'},
 {'id': '2', 'label': 'Glenbawn Dam'},
 {'id': '143', 'label': 'From Barnard'},
 {'id': '530', 'label': 'Losses associated with Barnard transfers'},
 {'id': '141', 'label': 'Mac Gen Barnard account'},
 {'id': '502', 'label': 'liddell PS pump - Barnard account'},
 {'id': '510', 'label': 'Glenbawn freeboard exc Barnard water'},
 {'id': '130', 'label': 'Account spill'},
 {'id': '138', 'label': 'Barnard spills to Hunter'},
 {'id': '140', 'label': 'Combines Hunter flows & Barnard account spills'},
 {'id': '152', 'label': 'Total orders to Glenbawn'},
 {'id': '3', 'label': 'Min Flo

In [8]:
from dash import Dash, html
import dash_cytoscape as cyto

app = Dash(__name__)

app.layout = html.Div([
    cyto.Cytoscape(
        id='cytoscape-two-nodes',
        layout={'name': 'preset'},
        style={'width': '100%',
               'height': '600px'},
        elements=elements+nodes

    )
])


if __name__ == '__main__':
    app.run(debug=True)

----
Fors input

In [29]:
input_fors_file=open("./tests/Test.fors",'r')
input_fors=input_fors_file.readlines()
input_fors_file.close()

In [30]:
out=[]
del_line=[]
for i,line in enumerate(input_fors): # check each line of input
    init=i
    temp=line.strip().split('+')[0] # split line based on '+' and keep first entry
    while '+' in input_fors[init]: # if that line had a '+', append the next line and remember the next line number to delete later
        temp=temp+input_fors[init+1].strip().split('+')[0]
        init+=1
        del_line.append(init)
    out.append(temp+'\n') # add the linebreak back in

In [31]:
unwrap_input=[line for i,line in enumerate(out) if i not in del_line]   # rewrite unwrapped lines and delete lines that were appended

In [32]:
fors_nodes=[[i,x.split('/')[0].strip('[]\n ')] for i,x in enumerate(unwrap_input) if '[' in x]  # excluding comments, find nodes based on containing "["

In [33]:
fors_nodes

[[0, 'input'],
 [4, 'input'],
 [8, 'input'],
 [12, 'input'],
 [16, 'input'],
 [20, 'input'],
 [24, 'input'],
 [30, 'inflow'],
 [35, 'gauge'],
 [43, 'inflow'],
 [48, 'gauge'],
 [56, 'inflow'],
 [61, 'gauge'],
 [69, 'node'],
 [73, 'nonlinear reach'],
 [78, 'gauge'],
 [83, 'sacramento'],
 [89, 'gauge'],
 [95, 'sacramento model'],
 [102, 'recorder'],
 [107, 'recorder'],
 [112, 'recorder'],
 [117, 'recorder']]

In [34]:
list=[]
for j in fors_nodes:
    init=j[0]   # line number of node section
    while unwrap_input[init] != '\n':   # keep reading lines below node header until you find an empty line
        init+=1
        if init == len(unwrap_input):
            break
    my_dict={x.split('=')[0].strip(): x.split('=')[1].strip() for x in unwrap_input[j[0]+1:init]} # assign node parameters to dictionary
    my_dict['node_type']=j[1]   # assign node type to dictionary
    list.append(my_dict)    # create list of dictionaries

In [35]:
# convert to cytoscape node format. Only include nodes that have a name (non-recorder) and are of a particular node type
nodes=[{'data': {'id': x['name'], 'label': x['name']}, 'classes': x['node_type']} for x in list if ('name' in x) and (x['node_type'] in ['inflow','gauge','node','nonlinear reach','sacramento'])]

In [36]:
# convert to cytoscape edge format. Only include nodes that have a 'dsnode'
edges=[{'data': {'source': x['name'], 'target': x['dsnode']}} for x in list if 'dsnode' in x]

In [37]:
from dash import Dash, html, dcc, Input, Output, callback
import dash_cytoscape as cyto

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown-update-layout',
        value='cose',
        clearable=False,
        options=[
            {'label': name.capitalize(), 'value': name}
            for name in ['cose', 'grid', 'random', 'circle', 'concentric']
        ]
    ),
    cyto.Cytoscape(
        id='cytoscape-update-layout',
        layout={'name': 'cose'},
        style={'width': '100%', 'height': '600px'},
        elements=edges+nodes,
        stylesheet=[
            {
                'selector': 'node',
                'style': {
                    'label': 'data(id)',
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'curve-style': 'bezier',
                    'line-color': 'black',
                    'mid-target-arrow-shape': 'vee',
                    'mid-target-arrow-color': 'black'
                }
            },
            {
                'selector': '.inflow',
                'style': {
                    'background-color': 'green'
                }
            },
            {
                'selector': '.gauge',
                'style': {
                    'background-color': 'pink'
                }
            },
                        {
                'selector': '.confluence',
                'style': {
                    'background-color': 'grey'
                }
            },
                        {
                'selector': '.nonlinear',
                'style': {
                    'background-color': 'red'
                }
            },
                        {
                'selector': '.sacramento',
                'style': {
                    'background-color': 'cyan'
                }
            }                         
        ]
    )
])

@callback(Output('cytoscape-update-layout', 'layout'),
              Input('dropdown-update-layout', 'value'))
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

if __name__ == '__main__':
    app.run(debug=True)

----
Folium test

In [61]:
input=pd.read_csv(filepath_or_buffer="./tests/35013f15b(GIS).csv",quotechar='\'')
input

,Reach,Branch,Link,Node number,Downstream node,Node type,Longitude,Latitude,Label,Associated node
0,Reach001,1,1,129,136,1.0,150.9459,-31.6886,Barnard River,0
1,Reach001,1,2,136,132,3.1,150.9402,-31.7452,Macquarie Gen transfer to Hunter,143
2,Reach001,2,1,131,132,1.0,151.0374,-31.7469,Orham Creek,0
3,Reach001,3,1,132,133,11.0,150.9761,-31.8026,Barnard R / Orham Ck,0
4,Reach001,3,2,133,134,0.0,150.9762,-31.8331,Barnard R GS for 80% flow,0
...,...,...,...,...,...,...,...,...,...,...
386,Reach096,64,1,678,679,1.2,152.0447,-33.1384,Receive transfers from Lostock,0
387,Reach096,64,2,679,684,2.1,152.1403,-33.1564,HWC Account in Glennies,0
388,Reach096,64,3,684,681,3.0,152.2281,-33.1456,Diversion from HWC Reserve,0
389,Reach096,65,1,681,610,11.0,152.3344,-33.1150,day count confluence,0


In [39]:
nodes=[{'data': {'id': str(input["Node number"][x]), 'label': input["Label"][x]}, 'position': {'x' : input["Longitude"][x], 'y': input["Latitude"][x]}} for x in range(len(input))]
nodes

[{'data': {'id': '129', 'label': 'Barnard River'},
  'position': {'x': 150.9459, 'y': -31.6886}},
 {'data': {'id': '136', 'label': 'Macquarie Gen transfer to Hunter'},
  'position': {'x': 150.9402, 'y': -31.7452}},
 {'data': {'id': '131', 'label': 'Orham Creek'},
  'position': {'x': 151.0374, 'y': -31.7469}},
 {'data': {'id': '132', 'label': 'Barnard R / Orham Ck'},
  'position': {'x': 150.9761, 'y': -31.8026}},
 {'data': {'id': '133', 'label': 'Barnard R GS for 80% flow'},
  'position': {'x': 150.9762, 'y': -31.8331}},
 {'data': {'id': '134', 'label': 'remove d/s flows'},
  'position': {'x': 150.9772, 'y': -31.8646}},
 {'data': {'id': '1', 'label': 'Glenbawn inflow'},
  'position': {'x': 151.0502, 'y': -31.8585}},
 {'data': {'id': '135', 'label': 'Barnard / Hunter'},
  'position': {'x': 151.005, 'y': -31.906}},
 {'data': {'id': '219', 'label': 'Glenbawn Irr'},
  'position': {'x': 150.9994, 'y': -31.9371}},
 {'data': {'id': '2', 'label': 'Glenbawn Dam'},
  'position': {'x': 150.9879, '

In [40]:
elements=[{'data': {'source': str(input["Node number"][x]), 'target': str(input["Downstream node"][x])}} for x in range(len(input)-1) if str(input["Downstream node"][x]) not in ['1008','1010','1014']]

In [41]:
elements

[{'data': {'source': '129', 'target': '136'}},
 {'data': {'source': '136', 'target': '132'}},
 {'data': {'source': '131', 'target': '132'}},
 {'data': {'source': '132', 'target': '133'}},
 {'data': {'source': '133', 'target': '134'}},
 {'data': {'source': '134', 'target': '135'}},
 {'data': {'source': '1', 'target': '135'}},
 {'data': {'source': '135', 'target': '219'}},
 {'data': {'source': '219', 'target': '2'}},
 {'data': {'source': '2', 'target': '140'}},
 {'data': {'source': '143', 'target': '530'}},
 {'data': {'source': '530', 'target': '141'}},
 {'data': {'source': '141', 'target': '502'}},
 {'data': {'source': '502', 'target': '510'}},
 {'data': {'source': '510', 'target': '130'}},
 {'data': {'source': '130', 'target': '138'}},
 {'data': {'source': '138', 'target': '140'}},
 {'data': {'source': '140', 'target': '152'}},
 {'data': {'source': '152', 'target': '3'}},
 {'data': {'source': '3', 'target': '215'}},
 {'data': {'source': '215', 'target': '4'}},
 {'data': {'source': '4',

In [42]:
[x["data"] for x in elements]

[{'source': '129', 'target': '136'},
 {'source': '136', 'target': '132'},
 {'source': '131', 'target': '132'},
 {'source': '132', 'target': '133'},
 {'source': '133', 'target': '134'},
 {'source': '134', 'target': '135'},
 {'source': '1', 'target': '135'},
 {'source': '135', 'target': '219'},
 {'source': '219', 'target': '2'},
 {'source': '2', 'target': '140'},
 {'source': '143', 'target': '530'},
 {'source': '530', 'target': '141'},
 {'source': '141', 'target': '502'},
 {'source': '502', 'target': '510'},
 {'source': '510', 'target': '130'},
 {'source': '130', 'target': '138'},
 {'source': '138', 'target': '140'},
 {'source': '140', 'target': '152'},
 {'source': '152', 'target': '3'},
 {'source': '3', 'target': '215'},
 {'source': '215', 'target': '4'},
 {'source': '4', 'target': '80'},
 {'source': '5', 'target': '220'},
 {'source': '220', 'target': '80'},
 {'source': '80', 'target': '81'},
 {'source': '6', 'target': '221'},
 {'source': '221', 'target': '81'},
 {'source': '81', 'targe

In [43]:
[x['position'] for x in nodes]

[{'x': 150.9459, 'y': -31.6886},
 {'x': 150.9402, 'y': -31.7452},
 {'x': 151.0374, 'y': -31.7469},
 {'x': 150.9761, 'y': -31.8026},
 {'x': 150.9762, 'y': -31.8331},
 {'x': 150.9772, 'y': -31.8646},
 {'x': 151.0502, 'y': -31.8585},
 {'x': 151.005, 'y': -31.906},
 {'x': 150.9994, 'y': -31.9371},
 {'x': 150.9879, 'y': -31.9617},
 {'x': 150.7929, 'y': -31.8018},
 {'x': 150.7626, 'y': -31.8476},
 {'x': 150.7714, 'y': -31.8755},
 {'x': 150.8053, 'y': -31.9015},
 {'x': 150.839, 'y': -31.9158},
 {'x': 150.8659, 'y': -31.9276},
 {'x': 150.8946, 'y': -31.9527},
 {'x': 150.9224, 'y': -31.9808},
 {'x': 150.9747, 'y': -32.0172},
 {'x': 151.0001, 'y': -32.0435},
 {'x': 151.0187, 'y': -32.0765},
 {'x': 151.0086, 'y': -32.1031},
 {'x': 151.0565, 'y': -32.1302},
 {'x': 151.0176, 'y': -32.1416},
 {'x': 150.9651, 'y': -32.1257},
 {'x': 150.9342, 'y': -32.03},
 {'x': 150.9367, 'y': -32.0746},
 {'x': 150.9219, 'y': -32.1339},
 {'x': 150.8271, 'y': -32.1563},
 {'x': 150.8481, 'y': -32.1506},
 {'x': 150.6299

In [44]:
node_list=[[x['data']['source'],x['data']['target']] for x in elements]

In [45]:
node_locations=[]
for i,j in node_list:
    for x in nodes:        
        if x['data']['id']==i:
            source=[x['position']['y'],x['position']['x']]
        if x['data']['id']==j:
            target=[x['position']['y'],x['position']['x']]
    node_locations.append([source,target])
            

In [46]:
node_locations

[[[-31.6886, 150.9459], [-31.7452, 150.9402]],
 [[-31.7452, 150.9402], [-31.8026, 150.9761]],
 [[-31.7469, 151.0374], [-31.8026, 150.9761]],
 [[-31.8026, 150.9761], [-31.8331, 150.9762]],
 [[-31.8331, 150.9762], [-31.8646, 150.9772]],
 [[-31.8646, 150.9772], [-31.906, 151.005]],
 [[-31.8585, 151.0502], [-31.906, 151.005]],
 [[-31.906, 151.005], [-31.9371, 150.9994]],
 [[-31.9371, 150.9994], [-31.9617, 150.9879]],
 [[-31.9617, 150.9879], [-31.9808, 150.9224]],
 [[-31.8018, 150.7929], [-31.8476, 150.7626]],
 [[-31.8476, 150.7626], [-31.8755, 150.7714]],
 [[-31.8755, 150.7714], [-31.9015, 150.8053]],
 [[-31.9015, 150.8053], [-31.9158, 150.839]],
 [[-31.9158, 150.839], [-31.9276, 150.8659]],
 [[-31.9276, 150.8659], [-31.9527, 150.8946]],
 [[-31.9527, 150.8946], [-31.9808, 150.9224]],
 [[-31.9808, 150.9224], [-32.0172, 150.9747]],
 [[-32.0172, 150.9747], [-32.0435, 151.0001]],
 [[-32.0435, 151.0001], [-32.0765, 151.0187]],
 [[-32.0765, 151.0187], [-32.1031, 151.0086]],
 [[-32.1031, 151.0086

In [47]:
longitude_bound=np.mean([x['position']['x'] for x in nodes])
latitude_bound=np.mean([x['position']['y'] for x in nodes])

In [48]:
import folium 

m = folium.Map(location=[latitude_bound,longitude_bound])

for i in node_locations:
    folium.PolyLine(i).add_to(m)
    folium.Marker(i[0]).add_to(m)

m

----
Folium from csv

In [49]:
import pandas as pd
import numpy as np

In [55]:
input=pd.read_csv(filepath_or_buffer="./tests/35013f15b(GIS).csv",quotechar='\'')
input


,Reach,Branch,Link,Node number,Downstream node,Node type,Longitude,Latitude,Label,Associated node
0,Reach001,1,1,129,136,1.0,150.9459,-31.6886,Barnard River,0
1,Reach001,1,2,136,132,3.1,150.9402,-31.7452,Macquarie Gen transfer to Hunter,143
2,Reach001,2,1,131,132,1.0,151.0374,-31.7469,Orham Creek,0
3,Reach001,3,1,132,133,11.0,150.9761,-31.8026,Barnard R / Orham Ck,0
4,Reach001,3,2,133,134,0.0,150.9762,-31.8331,Barnard R GS for 80% flow,0
...,...,...,...,...,...,...,...,...,...,...
386,Reach096,64,1,678,679,1.2,152.0447,-33.1384,Receive transfers from Lostock,0
387,Reach096,64,2,679,684,2.1,152.1403,-33.1564,HWC Account in Glennies,0
388,Reach096,64,3,684,681,3.0,152.2281,-33.1456,Diversion from HWC Reserve,0
389,Reach096,65,1,681,610,11.0,152.3344,-33.1150,day count confluence,0


In [56]:
temp=input["Node type"].unique()
temp.sort()
temp

array([ 0. ,  0.3,  0.5,  0.6,  1. ,  1.1,  1.2,  2. ,  2.1,  3. ,  3.1,
        3.2,  3.4,  3.6,  3.9,  4. ,  8. ,  8.2,  8.3,  9. ,  9.1, 11. ])

In [57]:
[str(x) for x in input["Node type"]]

['1.0',
 '3.1',
 '1.0',
 '11.0',
 '0.0',
 '4.0',
 '1.1',
 '11.0',
 '0.0',
 '2.1',
 '1.2',
 '4.0',
 '2.1',
 '3.2',
 '0.5',
 '9.0',
 '0.3',
 '11.0',
 '0.3',
 '9.0',
 '3.0',
 '0.3',
 '1.0',
 '0.0',
 '11.0',
 '1.0',
 '0.0',
 '11.0',
 '1.0',
 '0.0',
 '1.0',
 '0.0',
 '1.0',
 '0.0',
 '11.0',
 '0.5',
 '9.1',
 '0.3',
 '8.0',
 '8.0',
 '8.0',
 '8.0',
 '8.2',
 '1.0',
 '8.2',
 '8.2',
 '4.0',
 '0.0',
 '3.0',
 '8.2',
 '1.0',
 '8.0',
 '8.2',
 '8.2',
 '4.0',
 '0.0',
 '8.2',
 '8.0',
 '8.0',
 '8.0',
 '8.0',
 '8.0',
 '11.0',
 '11.0',
 '9.1',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '8.0',
 '0.3',
 '8.0',
 '0.3',
 '8.0',
 '3.2',
 '8.2',
 '1.2',
 '0.3',
 '8.2',
 '3.4',
 '0.3',
 '9.1',
 '0.5',
 '0.5',
 '3.1',
 '0.3',
 '8.2',
 '0.3',
 '3.9',
 '0.3',
 '1.0',
 '1.2',
 '1.2',
 '0.3',
 '1.0',
 '2.0',
 '3.0',
 '3.0',
 '3.0',
 '3.4',
 '3.0',
 '0.0',
 '3.1',
 '1.0',
 '1.2',
 '0.0',
 '0.0',
 '2.0',
 '0.0',
 '3.0',
 '3.4',
 '3.4',
 '3.0',
 '0.0',
 '0.0',
 '1.0',
 '1.2',
 '0.3',
 '0.0',
 '0.3',
 '1.2',
 '2.0',
 '11.0',
 '9.1',

In [58]:
import folium 
from folium.plugins import TagFilterButton

In [59]:
longitude_bound=input["Longitude"].mean()
latitude_bound=input["Latitude"].mean()

muted_rainbow = ["#72AC57", "#639188", "#5C77B0", "#765EA4", "#98467A", "#BA3438", "#CF512B", "#E18433"]

m = folium.Map(location=[latitude_bound,longitude_bound],tiles=None)

for idx in input.index:
    label=str(input["Label"][idx])
    number=str(input["Node number"][idx])
    type=str(input["Node type"][idx])
    lat=input["Latitude"][idx]
    lon=input["Longitude"][idx]
    
    tooltip="<b>Node:</b> " + number + "<br><b>Node type:</b> " + type + "<br><b>Label:</b> " + label
    
    folium.Marker([lat,lon],
                tooltip=tooltip,
                tags=[type],
                icon=folium.Icon(color="#72AC57",prefix="fa",icon="water")
                ).add_to(m)
    
    dsnode=input["Downstream node"][idx]
    if dsnode in input["Node number"].to_list():
        source = [lat,lon]
        target = [input[input["Node number"]==dsnode]["Latitude"].values[0],input[input["Node number"]==dsnode]["Longitude"].values[0]]
        
        folium.PolyLine([source,target]).add_to(m)

node_types=input["Node type"].unique()
node_types.sort()
categories = [str(x) for x in node_types]

folium.raster_layers.TileLayer(
    tiles="http://{s}.google.com/vt/lyrs=p&hl=en&x={x}&y={y}&z={z}",
    attr="google",
    name="Google Terrain",
    max_zoom=20,
    subdomains=["mt0", "mt1", "mt2", "mt3"],
    overlay=False,
    control=True,
).add_to(m)

folium.raster_layers.TileLayer(
    tiles="http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="google",
    name="Google Maps",
    max_zoom=20,
    subdomains=["mt0", "mt1", "mt2", "mt3"],
    overlay=False,
    control=True,
).add_to(m)

folium.raster_layers.TileLayer(
    tiles="http://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attr="google",
    name="Google Street View",
    max_zoom=20,
    subdomains=["mt0", "mt1", "mt2", "mt3"],
    overlay=False,
    control=True,
).add_to(m)

folium.LayerControl().add_to(m)

TagFilterButton(categories).add_to(m)

m

C:\Users\Joel Rutten\AppData\Local\Temp\ipykernel_13320\1205773476.py:20: UserWarning:

color argument of Icon should be one of: {'lightgray', 'orange', 'gray', 'white', 'pink', 'black', 'lightred', 'green', 'blue', 'lightgreen', 'purple', 'beige', 'darkpurple', 'darkgreen', 'red', 'darkblue', 'darkred', 'cadetblue', 'lightblue'}.



In [60]:
input["Node number"].to_list()

[129,
 136,
 131,
 132,
 133,
 134,
 1,
 135,
 219,
 2,
 143,
 530,
 141,
 502,
 510,
 130,
 138,
 140,
 152,
 3,
 215,
 4,
 5,
 220,
 80,
 6,
 221,
 81,
 7,
 222,
 20,
 223,
 145,
 224,
 82,
 208,
 8,
 209,
 9,
 85,
 87,
 113,
 10,
 11,
 104,
 105,
 12,
 13,
 14,
 254,
 15,
 16,
 106,
 17,
 18,
 19,
 95,
 88,
 114,
 115,
 116,
 117,
 83,
 84,
 21,
 22,
 23,
 24,
 25,
 26,
 160,
 312,
 161,
 118,
 651,
 27,
 28,
 200,
 256,
 280,
 342,
 142,
 171,
 170,
 164,
 158,
 96,
 150,
 29,
 151,
 30,
 31,
 172,
 508,
 155,
 32,
 35,
 92,
 93,
 201,
 33,
 205,
 36,
 44,
 45,
 504,
 91,
 46,
 90,
 110,
 204,
 230,
 47,
 101,
 206,
 48,
 34,
 512,
 501,
 513,
 231,
 232,
 202,
 290,
 37,
 291,
 233,
 38,
 39,
 40,
 123,
 125,
 41,
 103,
 42,
 313,
 119,
 120,
 43,
 49,
 292,
 50,
 51,
 107,
 52,
 682,
 149,
 683,
 686,
 672,
 687,
 53,
 693,
 54,
 694,
 671,
 688,
 689,
 690,
 691,
 613,
 216,
 55,
 56,
 57,
 102,
 97,
 99,
 112,
 58,
 59,
 98,
 122,
 100,
 257,
 283,
 153,
 60,
 293,
 284,
 281,
